// TODO
// примери за falsy и truthy стойности
// примери за ??
// примери за !!

Този бележник демонстрира прости асинхронни шаблони в JavaScript и малка имплементация на EventEmitter (ES5). Той отразява примерите от файловете в репото и ги групира в обяснителни секции.
Следвайте съдържанието: falsy/truthy, ??, !!, var срещу let hoisting, бележки за event loop, EventEmitter (ES5), използване на Node EventEmitter, promises.

## Съдържание
1. Falsy и Truthy стойности
2. Nullish coalescing `??`
3. Двойно отрицание `!!`
4. Hoisting на променливи: `var` срещу `let` в цикли с асинхронни callback-и
5. Бележки за event loop (`setTimeout`, `setImmediate`)
6. Изграждане на EventEmitter от нулата (ES5)
7. Използване на Node `EventEmitter` (пример от репото)
8. Пример с Promises
9. Обобщение и следващи стъпки

### 1 — Falsy и Truthy стойности
Често срещаните falsy стойности в JS: `false`, `0`, `''` (празен низ), `null`, `undefined`, `NaN`. Всичко останало е truthy.
Малките примери по-долу показват как стойностите се държат в условни изрази и при преобразуване към булев тип.

In [ ]:
// Примери за falsy и truthy
const vals = [false, 0, '', null, undefined, NaN, '0', 'hello', 1, {}];
vals.forEach(v => console.log(JSON.stringify(v), '->', Boolean(v)));

// Пример: поведение в условен израз
if (0) { console.log('0 is truthy'); } else { console.log('0 is falsy'); }
if ('hello') { console.log('is truthy'); } else { console.log('is falsy'); }

### 2 — Nullish coalescing (`??`)
Операторът `??` връща дясната страна, когато лявата страна е `null` или `undefined` (но не и други falsy стойности като `0` или `''`).

In [ ]:
console.log(null ?? 'по подразбиране'); // 'по подразбиране'
console.log(undefined ?? 'по подразбиране'); // 'по подразбиране'
console.log(0 ?? 'по подразбиране'); // 0 (не е nullish)
console.log('' ?? 'по подразбиране'); // '' (не е nullish)
// Сравнение с ||
console.log(0 || 'fallback'); // 'fallback' защото 0 е falsy
console.log(null || 'fallback'); // 'fallback'

### 3 — Двойно отрицание (`!!`)
`!!value` преобразува стойност към нейния булев еквивалент. Полезно за явни проверки за truthiness.

In [ ]:
console.log(!!0); // false
console.log(!!''); // false
console.log(!!'hello'); // true
console.log(!!{}); // true

### 4 — Hoisting на променливи: `var` срещу `let` в цикли
Този класически пример показва как `var` е function-scoped и създава проблеми със замиканията вътре в асинхронни callback-и, докато `let` е block-scoped и ги избягва.

In [ ]:
console.log('var цикъл:');
for (var i = 0; i < 3; i++) {
  setTimeout(() => {
    console.log('var i ->', i);
  }, 50);
}

console.log('let цикъл:');
for (let j = 0; j < 3; j++) {
  setTimeout(() => {
    console.log('let j ->', j);
  }, 100);
}

### 5 — Бележки за event loop (`setTimeout` vs `setImmediate`)
- `setTimeout(fn, 0)` планира макротаск след текущия call stack и всички микротаскове.
- `setImmediate` (в Node) планира callback за check фазата — обикновено се изпълнява след I/O събития и в много случаи има сходен ред с `setTimeout(fn, 0)`, но редът може да се различава.
Малки демонстрации по-долу използват и двата шаблона.

In [ ]:
console.log('start');
setTimeout(() => console.log('timeout 0'), 0);
if (typeof setImmediate === 'function') {
  setImmediate(() => console.log('setImmediate'));
} else {
  console.log('setImmediate не е наличен в тази среда');
}
Promise.resolve().then(() => console.log('microtask (promise)'));
console.log('end');

### 6 — Изграждане на EventEmitter от нулата (ES5)
По-долу е простата ES5 имплементация на EventEmitter от `event.emit.test.js`. Тя демонстрира `on()` и `emit()` и използва `setTimeout` вътре в `emit()` за асинхронно извикване на слушателите.
(Това е с учебна цел — не е заместител за продукционен код.)

In [ ]:
function EventEmitter() {
  this.events = {};
}

EventEmitter.prototype.on = function(eventName, fn) {
  if (!this.events[eventName]) {
    this.events[eventName] = [];
  }
  this.events[eventName].push(fn);
};

EventEmitter.prototype.off = function(eventName, fn) {
  var listeners = this.events[eventName];
  if (!listeners) return;
  for (var i = listeners.length - 1; i >= 0; i--) {
    if (listeners[i] === fn) {
      listeners.splice(i, 1);
    }
  }
};

EventEmitter.prototype.once = function(eventName, fn) {
  var self = this;
  function wrapper() {
    // извикваме оригиналния слушател с подадените аргументи
    fn.apply(null, arguments);
    // премахваме wrapper-а след първото извикване
    self.off(eventName, wrapper);
  }
  this.on(eventName, wrapper);
};

EventEmitter.prototype.emit = function(eventName) {
  var args = Array.prototype.slice.call(arguments, 1);
  var event = this.events[eventName];
  if (event) {
    event.forEach(function(fn) {
      setTimeout(function() {
        fn.apply(null, args);
      }, 0);
    });
  }
};

// Демонстрация на използване
var emitter = new EventEmitter();
console.log('настройка на събитията');

function listenerA() { console.log('listenerA извикан'); }
function listenerB() { console.log('listenerB извикан'); }

// добавяме два слушателя и после премахваме единия
emitter.on('test', listenerA);
emitter.on('test', listenerB);
console.log('emit 1 (трябва да се изпълнят и двата):');
emitter.emit('test');

// премахваме listenerA преди следващото emit
emitter.off('test', listenerA);
console.log('emit 2 (трябва да се изпълни само listenerB):');
emitter.emit('test');

// пример за once
emitter.once('onceEvent', function() { console.log('onceEvent задействано'); });
console.log('emit onceEvent първи път:');
emitter.emit('onceEvent');
console.log('emit onceEvent втори път (не трябва да се изпълни):');
emitter.emit('onceEvent');

// Използваме setImmediate ако е налично (в Node). В противен случай, emit вече е планирано с setTimeout вътре в emit().
if (typeof setImmediate === 'function') {
  setImmediate(function() { /* вече emit-нахме по-горе */ });
} else {
  // няма допълнително необходимо; emit-овете бяха планирани с setTimeout вътре в emit()
}

console.log('след демонстрационните emit-ове');

### 7 — Node `EventEmitter` (пример от `event.emit.js`)
Пример показва използване на вградения модул `events` в Node и изпращане на събития с аргументи. Ако стартирате този бележник в Node-базиран JS kernel, можете да `require('events')`. В противен случай, разгледайте го като демонстрация.

In [ ]:
// Пример за Node (изисква Node kernel).
try {
  const EventEmitter = require('events');
  const em = new EventEmitter();
  em.on('greet', function(name) {
    console.log(`Здравей, ${name}!`);
  });
  em.on('user', function(id, name, age) {
    console.log(`Потребител ${id}: ${name}, ${age} години`);
  });
  // изпращаме няколко събития след закъснение
  setTimeout(() => {
    em.emit('greet', 'Алиса');
    em.emit('user', 123, 'Боб', 25);
  }, 500);
} catch (err) {
  console.log('Примерът с вградения EventEmitter на Node е прескочен (не е наличен в този kernel).');
}

### 8 — Пример с Promises
Малък демо-пример с promise, копиран от `event.emit.test.js`. В оригиналния файл той се разрешава след 5 секунди; тук съкращаваме закъснението за интерактивна демонстрация.

In [ ]:
let p = new Promise((resolve, reject) => {
  setTimeout(() => {
    if (false) {
      reject('Promise отхвърлен!');
      return;
    }
    resolve('Promise разрешен!');
  }, 200); // скъсяваме за демото
});

p.then(message => console.log(message));
p.then(message => console.log('Втори then: ' + message));

### 9 — Обобщение и следващи стъпки
- Видяхте falsy/truthy, `??`, `!!`, var срещу let hoisting, реда на event loop, малък ES5 EventEmitter, използване на Node EventEmitter и пример с Promise.
- Следващи стъпки (опционално): добавяне на `off()` към ES5 emitter (вече е включено), демонстриране на обработка на грешки при emit, добавяне на кратки упражнения или конвертиране на ES5 класа в ES6 `class` синтаксис.